In [ ]:
import requests
from bs4 import BeautifulSoup

def search_amazon(product):
    url = f"https://www.amazon.in/s?k={product.replace(' ', '+')}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        products = soup.find_all("span", {"class": "a-size-medium a-color-base a-text-normal"})
        
        if products:
            print(f"Products found for '{product}':\n")
            for idx, item in enumerate(products, 1):
                print(f"{idx}. {item.text.strip()}")
        else:
            print(f"No products found for '{product}'.")
    else:
        print("Failed to fetch data from Amazon.")

if __name__ == "__main__":
    user_input = input("Enter the product you want to search for on Amazon.in: ").strip()
    search_amazon(user_input)


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_product_details(product_url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
    response = requests.get(product_url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        brand = soup.find("a", {"id": "bylineInfo"})
        brand_name = brand.text.strip() if brand else "-"
        product_title = soup.find("span", {"id": "productTitle"}).text.strip()
        price = soup.find("span", {"id": "priceblock_ourprice"})
        price = price.text.strip() if price else "-"
        return_exchange = soup.find("div", {"id": "RETURNS_POLICY"})
        return_exchange = return_exchange.text.strip() if return_exchange else "-"
        expected_delivery = soup.find("span", {"id": "ddmDeliveryMessage"})
        expected_delivery = expected_delivery.text.strip() if expected_delivery else "-"
        availability = soup.find("div", {"id": "availability"})
        availability = availability.text.strip() if availability else "-"
        return brand_name, product_title, price, return_exchange, expected_delivery, availability, product_url
    else:
        return "-", "-", "-", "-", "-", "-", "-"

def search_amazon(product):
    products_data = []
    for page in range(1, 4):  # Scraping first 3 pages
        url = f"https://www.amazon.in/s?k={product.replace(' ', '+')}&page={page}"
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"}
        
        response = requests.get(url, headers=headers)
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, "html.parser")
            products = soup.find_all("span", {"class": "a-size-medium a-color


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import requests
import os

def scrape_images(keyword, num_images):
    driver = webdriver.Chrome(executable_path="path_to_chromedriver") # Change this to your ChromeDriver path
    driver.get("https://www.google.com/imghp?hl=en")

    search_box = driver.find_element_by_css_selector("input.gLFyf.gsfi")
    search_box.send_keys(keyword)
    search_box.send_keys(Keys.RETURN)

    # Scroll to load more images
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Get the page source and close the driver
    page_source = driver.page_source
    driver.close()

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')
    img_tags = soup.find_all('img', class_='rg_i')

    # Create a directory to save images
    if not os.path.exists(keyword):
        os.makedirs(keyword)

    # Download images
    for i, img_tag in enumerate(img_tags[:num_images], 1):
        img_url = img_tag['src']
        img_name = f"{keyword}_{i}.jpg"
        img_path = os.path.join(keyword, img_name)
        try:
            response = requests.get(img_url)
            with open(img_path, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded {img_name}")
        except Exception as e:
            print(f"Error downloading {img_name}: {e}")

if __name__ == "__main__":
    keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
    num_images_per_keyword = 10

    for keyword in keywords:
        scrape_images(keyword, num_images_per_keyword)


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_smartphones(keyword):
    url = f"https://www.flipkart.com/search?q={keyword.replace(' ', '%20')}"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        products = soup.find_all('div', {'class': '_1AtVbE'})

        data = []
        for product in products:
            brand_name = product.find('div', {'class': '_4rR01T'}).text
            smartphone_name = product.find('a', {'class': 'IRpwTa'}).text
            product_url = 'https://www.flipkart.com' + product.find('a', {'class': 'IRpwTa'})['href']

            specifications = product.find_all('li', {'class': 'rgWa7D'})
            specs_dict = {spec.find('span', {'class': 'IRpwTa'}).text: spec.find('span', {'class': '_2LYh3d'}).text
                          for spec in specifications}

            # Fill in missing details with '-'
            for key in ['Colour', 'RAM', 'Storage(ROM)', 'Primary Camera', 'Secondary Camera',
                        'Display Size', 'Battery Capacity']:
                specs_dict.setdefault(key, '-')

            price = product.find('div', {'class': '_30jeq3'}).text

            data.append({
                'Brand Name': brand_name,
                'Smartphone Name': smartphone_name,
                'Colour': specs_dict['Colour'],
                'RAM': specs_dict['RAM'],
                'Storage(ROM)': specs_dict['Storage(ROM)'],
                'Primary Camera': specs_dict['Primary Camera'],
                'Secondary Camera': specs_dict['Secondary Camera'],
                'Display Size': specs_dict['Display Size'],
                'Battery Capacity': specs_dict['Battery Capacity'],
                'Price': price,
                'Product URL': product_url
            })

        return data
    else:
        print("Failed to fetch data from Flipkart.")
        return []

if __name__ == "__main__":
    keyword = input("Enter the smartphone you want to search for on Flipkart: ").strip()
    data = scrape_smartphones(keyword)

    if data:
        df = pd.DataFrame(data)
        df.to_csv(f"{keyword}_smartphones.csv", index=False)
        print(f"Data saved to {keyword}_smartphones.csv successfully.")
    else:
        print("No data scraped.")


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

def scrape_coordinates(city):
    # Launch the Chrome browser
    driver = webdriver.Chrome(executable_path="path_to_chromedriver") # Change this to your ChromeDriver path
    driver.get("https://www.google.com/maps")

    # Find the search box, enter the city name, and submit the search
    search_box = driver.find_element_by_css_selector("input.tactile-searchbox-input")
    search_box.send_keys(city)
    search_box.send_keys(Keys.ENTER)

    # Wait for the page to load
    time.sleep(5)

    # Get the page source
    page_source = driver.page_source

    # Close the browser
    driver.quit()

    # Parse the HTML with BeautifulSoup
    soup = BeautifulSoup(page_source, 'html.parser')

    # Find the latitude and longitude from the page
    try:
        # Latitude and Longitude are stored in the URL of the map
        map_url = soup.find('meta', {'itemprop': 'image'})['content']
        lat_long_str = map_url.split('@')[1].split(',')[0:2]
        latitude = float(lat_long_str[0])
        longitude = float(lat_long_str[1])
        return latitude, longitude
    except Exception as e:
        print(f"Error: {e}")
        return None, None

if __name__ == "__main__":
    city = input("Enter the name of the city to get its coordinates: ")
    latitude, longitude = scrape_coordinates(city)
    if latitude is not None and longitude is not None:
        print(f"Coordinates of {city}: Latitude - {latitude}, Longitude - {longitude}")
    else:
        print(f"Failed to fetch coordinates for {city}")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_gaming_laptops():
    url = "https://www.digit.in/top-products/best-gaming-laptops-40.html"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        laptops = soup.find_all('div', class_='TopNumbeHeading sticky-footer')
        
        data = []
        for laptop in laptops:
            name = laptop.find('div', class_='TopNumbeHeading sticky-footer').text.strip()
            specs = laptop.find_all('div', class_='right-container')
            processor = specs[0].text.strip()
            memory = specs[1].text.strip()
            os = specs[2].text.strip()
            display = specs[3].text.strip()
            price = specs[4].text.strip()

            data.append({
                'Name': name,
                'Processor': processor,
                'Memory': memory,
                'Operating System': os,
                'Display': display,
                'Price': price
            })

        return data
    else:
        print("Failed to fetch data from digit.in.")
        return []

if __name__ == "__main__":
    data = scrape_gaming_laptops()

    if data:
        df = pd.DataFrame(data)
        df.to_csv("best_gaming_laptops.csv", index=False)
        print("Data saved to best_gaming_laptops.csv successfully.")
    else:
        print("No data scraped.")


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_billionaires():
    url = "https://www.forbes.com/billionaires/"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
    }

    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        billionaires = soup.find_all('div', class_='personName')

        data = []
        for billionaire in billionaires:
            rank = billionaire.find_previous('div', class_='rank').text.strip()
            name = billionaire.text.strip()
            net_worth = billionaire.find_next('div', class_='netWorth').text.strip()
            age = billionaire.find_next('div', class_='age').text.strip()
            citizenship = billionaire.find_next('div', class_='countryOfCitizenship').text.strip()
            source = billionaire.find_next('div', class_='source-column').text.strip()
            industry = billionaire.find_next('div', class_='category').text.strip()

            data.append({
                'Rank': rank,
                'Name': name,
                'Net Worth': net_worth,
                'Age': age,
                'Citizenship': citizenship,
                'Source': source,
                'Industry': industry
            })

        return data
    else:
        print("Failed to fetch data from Forbes.com.")
        return []

if __name__ == "__main__":
    data = scrape_billionaires()

    if data:
        df = pd.DataFrame(data)
        df.to_csv("billionaires.csv", index=False)
        print("Data saved to billionaires.csv successfully.")
    else:
        print("No data scraped.")


In [ ]:
from googleapiclient.discovery import build
from datetime import datetime

def get_video_comments(video_id):
    # Set up the YouTube Data API service
    youtube = build('youtube', 'v3', developerKey='YOUR_API_KEY')

    # Request comments for the specified video
    response = youtube.commentThreads().list(
        part='snippet',
        videoId=video_id,
        maxResults=100,  # Max results per page (100 is the maximum allowed)
        order='time'     # Order comments by time
    ).execute()

    comments = []
    while response:
        for item in response['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comment_time = datetime.strptime(comment['publishedAt'], '%Y-%m-%dT%H:%M:%SZ')
            comments.append({
                'Comment': comment['textDisplay'],
                'Upvotes': comment['likeCount'],
                'Time': comment_time
            })

        # Check if there are more comments available
        if 'nextPageToken' in response:
            response = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                maxResults=100,
                pageToken=response['nextPageToken'],
                order='time'
            ).execute()
        else:
            break

    return comments

if __name__ == '__main__':
    video_id = input("Enter the YouTube video ID: ")
    all_comments = get_video_comments(video_id)

    if all_comments:
        print(f"Total comments extracted: {len(all_comments)}")
        for comment in all_comments:
            print(f"Comment: {comment['Comment']}")
            print(f"Upvotes: {comment['Upvotes']}")
            print(f"Time: {comment['Time']}")
            print("-------------------------------------")
    else:
        print("No comments found for the given video ID.")
